In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

mm = 0.1/2.54

# Global params (used by everyone else)
from matplotlib import rc
 
fontSize = 12
fontSizeLegend = 9
fontsizetext = 14 ## For a) b) c) d) etc.
lineWidth = 1.5
tickPad = 8
tickPad2 = 8
labelPadY = 6
labelPadX = 6
boxPad = 1
darkBlue = (0.0, 0.129, 0.2784)
darkRed = (0.7176, 0.0705, 0.207)

plt.rcParams["font.family"] = "Times New Roman"
plt.rc('axes', titlesize=fontSize)
plt.rc('axes', labelsize=fontSize)
plt.rc('xtick', labelsize=fontSize)
plt.rc('ytick', labelsize=fontSize)
plt.rc('legend', fontsize=fontSizeLegend)
plt.rc('lines', linewidth=lineWidth)
 
plt.rcParams['axes.labelpad'] = labelPadY
plt.rcParams['xtick.major.pad'] = tickPad
plt.rcParams['ytick.major.pad'] = tickPad
plt.rcParams['xtick.minor.pad'] = tickPad2
plt.rcParams['ytick.minor.pad'] = tickPad2
plt.rcParams['legend.borderaxespad'] = boxPad
 
plt.rcParams["mathtext.fontset"] = 'cm'

plt.rcParams['axes.linewidth'] = 1.5

In [2]:
CASE_PATH = '.'

def get_exp_df(loc):
    path = "validation/data/" + loc + ".xy"
    return pd.read_csv(path, header=0, sep=' ', index_col=0)

def get_sim_df(case_path, time, loc):
    path_sim = os.path.join(case_path, 'postProcessing/sampleFn/gas/',time, loc + ".xy")
    dat = np.loadtxt(path_sim, skiprows=1)
    sim_df = pd.DataFrame(dat, columns=['x','Y_H2','Y_H2O','Y_N2','Y_O2','Y_OH','T', 'Ux', 'Uy', 'Uz'])
    sim_df.set_index('x', inplace=True)
    return sim_df

def get_all_sim_dfs(case_path, time, locations):
    return {loc: get_sim_df(case_path, time, loc) for loc in locations}

# get_sim_df(CASE_PATH, '0.62', 'R_x1')
# get_all_sim_dfs(CASE_PATH, '0.62', ['R_x1', 'R_x5', 'R_x25', 'R_x50', 'ax'])

## Radial profiles of species and temperature

In [ ]:
time = '0.62'

xlabel = '$r~[\mathrm{mm}]$'
ylabel = '$Y_\mathrm{i}~[-]$'
ylabel2 = '$T~[\mathrm{K}]$'

case_path = CASE_PATH
locations = ['R_x1', 'R_x5', 'R_x25', 'R_x50', 'ax']
markersize = 2
styles = {
    'Y_H2':  dict(color='g',          label='H2',  linestyle="None", marker='^', markersize=markersize, fillstyle='full'),
    'Y_H2O': dict(color=darkRed,      label='H2O', linestyle="None", marker='o', markersize=markersize, fillstyle='none'),
    'Y_N2':  dict(color='m',          label='N2',  linestyle="None", marker='s', markersize=markersize, fillstyle='none'),
    'Y_O2':  dict(color='tab:orange', label='O2',  linestyle="None", marker='D', markersize=markersize, fillstyle='none'),
    'Y_OH':  dict(color='b',          label='OH',  linestyle="None", marker='x', markersize=markersize, fillstyle='none'),
    # 'T':     
}
styles_sim = {
    'Y_H2':  dict(color='g',          label='H2'),
    'Y_H2O': dict(color=darkRed,      label='H2O'),
    'Y_N2':  dict(color='m',          label='N2'),
    'Y_O2':  dict(color='tab:orange', label='O2'),
    'Y_OH':  dict(color='b',          label='OH'),
    # 'T':     
}
styleT = dict(color='k', label='T', linestyle="None", marker='+', markersize=1+markersize, fillstyle='none')
styleT_sim = dict(color='k', label='T')

def plot_radial(ax, axT, case_path, loc, xticks=None, y2ticks=None, xlabel=None, ylabel=None, ylabel2=None, time_str=None):
    print(loc)
    exp_df = get_exp_df(loc)
    if time_str is not None:
        sim_df = get_sim_df(case_path, time_str, loc)

    lines = []
    for style in styles:
        k = 25 if style=='Y_OH' else 1
        ax.plot(exp_df.index, k*exp_df[style], **styles[style])
        if time_str is not None:
            lines.append(ax.plot(sim_df.index*1000, k*sim_df[style], **styles_sim[style]))
    
    axT.plot(exp_df.index, exp_df["T"], **styleT)
    if time_str is not None:
        lines.append(axT.plot(sim_df.index*1000, sim_df["T"], **styleT_sim))
    
    return lines


fig = plt.figure(figsize=(160*mm, 119*mm))
axs = fig.subplots(2, 2, sharex=True)
axsT = [[axs[i][j].twinx() for j in range(2)] for i in range(2)]

lines_R_x1  = plot_radial(axs[0][0], axsT[0][0], case_path, 'R_x1' , time_str=time)
lines_R_x5  = plot_radial(axs[0][1], axsT[0][1], case_path, 'R_x5' , time_str=time)
lines_R_x25 = plot_radial(axs[1][0], axsT[1][0], case_path, 'R_x25', time_str=time)
lines_R_x50 = plot_radial(axs[1][1], axsT[1][1], case_path, 'R_x50', time_str=time)

for j in range(2):
    # all
    for i in range(2):
        axsT[i][j].set(
            ylim=[0, 2500]
        )
        axs[i][j].set(
            xlim = [0, 8], # 0,110 if loc=='ax'
            ylim = [0, 1],
        )
    # left column
    axs[j][0].set(
        yticks      = [0,0.2,0.4,0.6,0.8,1.0],
        yticklabels = [0,0.2,0.4,0.6,0.8,1.0],
        ylabel = ylabel,
    )
    axsT[j][0].set(
        yticks = [],
        yticklabels = [],
    )
    # right column
    axs[j][1].set(
        yticks = [],
        yticklabels = [],
    )
    axsT[j][1].set(
        yticks      = [0,500,1000,1500,2000,2500],
        yticklabels = [0,500,1000,1500,2000,2500],
        ylabel = ylabel2,
    )
    # bottom row
    axs[1][j].set(
        xticks      = [0,2,4,6,8],
        xticklabels = [0,2,4,6,8],
        xlabel = xlabel,
    )

axs[0][0].annotate(r'$\mathrm{H_2}$',    xy=(0.2,0.9),  xytext=(1.2, 0.92), xycoords='data',arrowprops=dict(arrowstyle="->",connectionstyle="arc"))
axs[0][0].annotate(r'$\mathrm{H_2O}$',   xy=(1.3,0.25), xytext=(1.0, 0.45), xycoords='data',arrowprops=dict(arrowstyle="->",connectionstyle="arc"))
axs[0][0].annotate(r'$\mathrm{N_2}$',    xy=(4.5, 0.75),  xytext=(4.5, 0.88), xycoords='data',arrowprops=dict(arrowstyle="->",connectionstyle="arc"))
axs[0][0].annotate(r'$\mathrm{O_2}$',    xy=(7,0.22),   xytext=(7.1, 0.35), xycoords='data',arrowprops=dict(arrowstyle="->",connectionstyle="arc"))
axs[0][0].annotate(r'$\mathrm{OH}\times 25$', xy=(1.9,0.1), xytext=(1.6, 0.35),  xycoords='data',arrowprops=dict(arrowstyle="->",connectionstyle="arc", relpos=(0.3, 0.5)))
axs[0][0].annotate(r'$T$',     xy=(4.1,0.6),    xytext=(5.3, 0.6),  xycoords='data',arrowprops=dict(arrowstyle="->",connectionstyle="arc"))

# props = dict(boxstyle='round', facecolor='gray', alpha=0.1)
props = None
axs[0][0].text(0.965, 0.965, r'$z = 1 \/ \mathrm{mm}$',  transform=axs[0][0].transAxes, verticalalignment='top', horizontalalignment='right', fontsize=9, bbox=props)
axs[0][1].text(0.965, 0.965, r'$z = 5 \/ \mathrm{mm}$',  transform=axs[0][1].transAxes, verticalalignment='top', horizontalalignment='right', fontsize=9, bbox=props)
axs[1][0].text(0.28,  0.965, r'$z = 25 \/ \mathrm{mm}$', transform=axs[1][0].transAxes, verticalalignment='top', horizontalalignment='right', fontsize=9, bbox=props)
axs[1][1].text(0.28,  0.965, r'$z = 50 \/ \mathrm{mm}$', transform=axs[1][1].transAxes, verticalalignment='top', horizontalalignment='right', fontsize=9, bbox=props)
fig.tight_layout()
fig.savefig(f"validation_profiles.png", dpi=300)

## Additional unpublished Figure with axial jet velocity profile at the exit of the nozzle

In [ ]:
time = '0.62'
case_path = CASE_PATH

sim_df = get_sim_df(case_path, time, 'inlet')

path_exp = "validation/data/inlet.xy"
exp_df = pd.read_csv(path_exp, header=0, sep=' ', index_col=0)


fig, ax = plt.subplots(figsize=[7,5], dpi=100)
ax.set_xlim([0,0.5])
ax.set_ylim([0,80])
ax.set_xlabel('Radial distance (mm)')
ax.set_ylabel('Axial velocity (m/s)')
axT = ax.twinx()
axT.set_ylabel('Temperature (K)')
axT.set_ylim([200, 600])

ax.plot(exp_df.index, exp_df['V_var_wall_T'], 'k-', label="U, Cheng et al.: var. T")
ax.plot(exp_df.index, exp_df['V_const_wall_T'], 'k--', label="U, Cheng et al.: const. T")
ax.plot(sim_df.index*1000, sim_df['Ux'], 'g-', label="U, OpenFOAM")

axT.plot(exp_df.index, exp_df["T_var_wall_T"], 'r-', label="T, Cheng et al.: var. T")
axT.plot(exp_df.index, exp_df["T_const_wall_T"], 'r--', label="T, Cheng et al.: const. T")
axT.plot(sim_df.index*1000, sim_df["T"], 'b', label="T, OpenFOAM")
ax.legend(loc=(0.45,0.83))
axT.legend(loc=(0.01,0.31))
# fig.savefig('inlet.pdf')